In [19]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

"""
This notebook just does everything in ./sh/preprocess.sh script
in an interactive manner.
"""

import sys
import os

sys.path.append('../')
sys.path.append('../paretoviz')
from paretoviz.utils import fmt
from paretoviz.utils import vectorops as vops

from paretoviz import normalizer as nrm
from paretoviz import tradeoff_calculator as tro
from paretoviz import peeler as plr
from paretoviz import palettizer as pltz

# Do all these plots with centroid based coloring

# rawfpath = "../data/spherical/spherical-3d.out"
# rawfpath = "../data/spherical/spherical-4d.out"
# rawfpath = "../data/spherical/spherical-8d.out"

# rawfpath = "../data/knee/knee-3d.out"
# rawfpath = "../data/knee/knee-4d.out"
# rawfpath = "../data/knee/knee-8d.out"

# rawfpath = "../data/line-surface/line-3d.out"
# rawfpath = "../data/line-surface/line-4d.out"
# rawfpath = "../data/line-surface/line-6d.out" # fails
# rawfpath = "../data/line-surface/line-8d.out"

# Do all these plots with constraint based coloring

# rawfpath = "../data/knee-const/knee-const-3d.out"
# rawfpath = "../data/knee-const/knee-const-4d.out"
# rawfpath = "../data/knee-const/knee-const-8d.out"

# rawfpath = "../data/knee-const-mod/knee-const-mod-3d.out"
# rawfpath = "../data/knee-const-mod/knee-const-mod-4d.out"

# rawfpath = "../data/isolated/isolated-3d.out"
# rawfpath = "../data/isolated/isolated-4d.out"
rawfpath = "../data/isolated/isolated-8d.out"

# rawfpath = "../data/c2dtlz2/c2dtlz2-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-4d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-5d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-8d.out"

# rawfpath = "../data/c2dtlz2/c2dtlz2-c1/c2dtlz2-c1-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-c2/c2dtlz2-c2-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-c3/c2dtlz2-c3-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-c4/c2dtlz2-c4-3d.out"

# some engineering design problems

# rawfpath = "../data/carcrash/carcrash-3d.out"
# rawfpath = "../data/carcrash/carcrash-c1/carcrash-c1-3d.out"
# rawfpath = "../data/carcrash/carcrash-c2/carcrash-c2-3d.out"

# rawfpath = "../data/gaa-das/gaa-das-10d.out"
# rawfpath = "../data/gaa-lhs/gaa-lhs-10d.out"

# some ML data sets

# rawfpath = "../data/airofoil/airofoil-5d.out"
# rawfpath = "../data/banknote/banknote-4d.out"
# rawfpath = "../data/blood/blood-4d.out" # fails
# rawfpath = "../data/cccp/cccp-4d.out"
# rawfpath = "../data/concrete/concrete-8d.out" # fails
# rawfpath = "../data/iris/iris-4d.out" 
# rawfpath = "../data/mammogram/mammogram-5d.out" # fails
# rawfpath = "../data/wil/wil-7d.out" # fails
# rawfpath = "../data/yeast/yeast-8d.out" #  fails 

path, rawfname = os.path.split(rawfpath)
normfname = rawfname.split('.')[0] + "-norm.out"
mufname = normfname.split('.')[0] + "-mu.out"

In [20]:
# This part does the normalizer.py operation

vals = fmt.load(rawfpath)

print("Normalizing {0:d} data points.".format(len(vals)))
vals_ = nrm.normalize(vals)
fmt.cat(vals_)

path, rawfile = os.path.split(rawfpath)
normfile = os.path.join(path, rawfile.split('.')[0] + "-norm.out")
print("Saving normalized data into {0:s} ...".format(normfile))
fmt.save(vals_, normfile)

Normalizing 10679 data points.
1.2936e-02	2.8019e-03	2.9505e-01	2.9210e-01	4.0759e-01	1.3925e-01	7.2622e-01	3.7341e-01
2.7346e-02	5.4074e-03	5.5160e-03	1.1669e-02	2.4543e-02	1.6553e-02	6.9658e-01	7.1647e-01
3.4832e-02	2.2061e-02	5.8794e-02	2.4193e-02	7.1032e-02	9.7029e-02	4.9318e-02	9.8926e-01
1.8030e-03	1.5195e-04	2.7342e-04	1.7498e-02	1.2023e-01	3.2157e-02	9.8859e-02	9.8739e-01
6.8459e-03	1.9604e-02	3.2771e-03	6.9843e-03	5.6255e-03	2.1163e-02	1.7465e-01	9.8418e-01
5.5143e-02	5.2661e-03	5.4063e-02	1.5311e-02	8.2804e-03	1.8950e-01	6.9148e-01	6.9391e-01
3.4220e-01	2.7926e-01	2.2630e-01	7.8804e-01	1.1883e-01	4.6222e-01	1.7649e-01	6.1956e-02
1.4300e-01	2.2716e-02	1.4161e-01	1.3535e-01	3.0357e-01	5.9429e-01	5.5832e-01	4.4895e-01
1.5571e-01	2.0371e-01	9.3791e-02	7.5862e-02	7.6413e-01	1.0610e-01	1.8195e-01	5.7354e-01
2.0773e-01	1.9380e-01	2.3292e-01	4.7276e-02	2.6748e-01	7.9257e-01	1.2605e-01	4.2837e-01
1.9914e-03	2.8889e-03	7.0895e-03	2.3998e-02	5.8028e-03	3.2567e-02	9.1870e-01	3.9293e-01
4

2.2590e-01	1.2343e-01	7.3103e-02	4.3556e-01	5.5027e-02	1.8985e-01	5.8617e-01	6.3191e-01
2.6839e-03	5.5222e-03	3.3024e-02	2.2135e-02	7.1738e-02	3.0479e-01	7.5435e-01	5.7640e-01
1.4971e-03	4.2464e-03	1.5788e-02	6.3820e-02	1.5105e-02	1.2482e-01	9.0044e-01	4.1189e-01
1.4498e-04	5.4388e-05	3.3373e-05	1.6102e-04	2.5246e-03	2.5766e-02	8.3747e-02	9.9615e-01
2.4913e-01	8.4852e-02	8.5910e-02	1.1132e-01	3.4011e-02	1.7547e-01	7.1597e-01	6.2887e-01
3.0481e-01	3.1438e-02	1.3230e-01	4.1694e-01	6.5805e-01	4.9259e-01	3.1218e-01	1.2486e-01
1.7795e-02	4.6509e-03	2.7848e-03	5.1036e-02	4.5629e-02	3.0095e-02	7.3977e-02	9.9451e-01
1.4607e-04	1.9799e-03	1.4583e-03	6.2680e-03	1.2101e-02	2.3127e-02	1.4628e-01	9.8888e-01
5.5587e-02	8.7958e-03	3.7189e-01	1.1408e-01	4.1083e-01	3.7970e-01	5.5549e-01	4.9989e-01
7.8628e-03	8.2420e-03	9.2882e-03	4.8697e-01	1.6132e-02	8.2099e-01	7.1920e-02	3.3300e-01
8.3293e-02	1.7705e-02	1.1927e-01	1.0159e-01	2.6606e-01	8.7479e-01	3.7354e-01	7.1700e-02
6.4737e-02	6.7240e-02	2.3126e-02

5.7215e-02	3.4707e-01	5.2230e-01	3.1101e-05	2.7584e-01	5.6353e-01	2.4442e-01	4.5497e-01
4.9006e-02	8.6934e-03	1.5427e-01	3.2429e-01	7.0078e-01	8.2932e-02	6.2940e-01	7.3493e-02
2.2697e-02	3.3128e-01	4.5513e-01	1.4379e-01	3.8089e-01	3.4140e-01	3.2048e-01	5.8906e-01
4.6882e-03	4.2137e-03	5.0923e-04	6.7419e-03	2.4361e-03	1.4304e-02	9.9401e-02	9.9491e-01
2.4089e-03	1.3310e-03	2.1070e-03	3.4608e-02	1.4825e-02	7.3804e-02	7.8292e-01	6.1673e-01
7.5950e-02	6.0326e-02	4.4313e-02	7.2578e-02	6.3421e-01	2.6409e-01	5.0992e-01	5.1863e-01
4.8020e-03	7.1094e-02	6.0733e-02	2.4005e-01	1.9468e-01	1.6965e-01	6.7416e-01	6.4874e-01
5.6431e-02	2.2700e-03	3.7487e-02	2.6277e-03	4.1263e-02	2.1968e-01	7.9793e-01	5.5732e-01
6.5156e-04	4.9053e-04	3.7650e-03	2.0656e-03	1.1043e-02	4.2324e-04	6.6172e-03	9.9991e-01
1.6564e-02	7.3330e-02	1.4347e-01	5.3399e-01	8.2750e-01	6.1280e-02	1.5553e-01	1.7562e-01
1.0178e-02	1.8675e-02	4.6349e-02	4.4303e-02	5.8198e-03	2.0375e-02	1.1503e-01	9.9112e-01
7.2419e-03	6.6640e-03	4.6999e-03

4.6274e-02	5.5041e-02	1.0531e-02	8.4159e-02	1.1121e-01	7.2047e-01	2.7288e-01	6.2227e-01
3.7288e-01	2.2620e-01	6.2861e-02	2.1932e-01	5.7346e-02	2.3315e-01	6.8328e-01	5.5136e-01
1.9913e-01	2.3288e-01	4.4364e-01	1.6326e-01	1.3783e-01	5.8641e-01	2.7781e-01	5.4431e-01
1.0121e-01	1.2086e-01	1.6887e-01	3.6290e-02	6.7897e-02	5.6240e-02	8.6921e-01	4.3844e-01
5.9465e-03	4.6024e-03	7.1598e-03	3.5711e-04	9.4337e-03	1.0898e-02	2.3783e-03	9.9985e-01
3.6210e-01	9.0366e-02	2.8045e-01	1.9734e-01	5.6540e-01	3.5058e-01	6.0186e-01	1.3623e-01
1.4183e-02	1.8405e-02	4.2693e-01	3.0689e-01	8.9486e-02	2.8731e-02	4.4891e-01	7.3798e-01
2.2515e-02	1.9393e-03	1.7537e-03	1.6329e-02	1.4252e-03	9.5420e-01	1.6187e-01	2.6092e-01
2.2541e-03	2.8633e-03	3.3323e-02	1.8296e-02	2.6995e-02	6.9237e-01	7.0090e-01	1.7379e-01
2.0242e-01	4.0186e-04	1.7395e-01	4.7326e-02	1.2223e-01	4.1702e-01	8.2211e-01	2.9016e-01
1.2295e-02	2.6225e-03	1.5386e-02	1.2584e-02	2.1920e-02	6.8243e-03	4.4405e-02	9.9853e-01
1.1996e-01	2.1935e-01	1.3667e-01

In [22]:
# This part does the tradeoff_calculator.py operation

# epsilon = 0.33 # spherical
# epsilon = 0.25 # knee
# epsilon = 0.025 # line
# epsilon = 0.125 # knee-const, mod
# epsilon = 0.33 # isolated-3d, 4d
epsilon = 0.025 # isolated-8d
# epsilon = 0.33 # c2dtlz2-3d, 4d
# epsilon = 0.15 # c2dtlz2-5d
# epsilon = 0.25 # carcrash

normfpath = os.path.join(path, normfname)
if os.path.exists(normfpath):
    points = fmt.load(normfpath)

    mu = tro.compute_tradeoff(points, epsilon, normalize = False)
    fmt.cat(mu)

    path, normfname = os.path.split(normfpath)
    mufpath = os.path.join(path, normfname.split('.')[0] + "-mu.out")
    print("Saving tradeoff values to {0:s} ...".format(mufpath))
    fmt.save(mu, mufpath)
else:
    print("Error: " + normfpath + " does not exist.")

Computing neighborhood ...
Computing tradeoff values with epsilon = 0.0250...
3.0938e-01
8.5926e-02
2.3798e-01
4.9956e-01
2.1100e-02
2.3767e-01
2.4612e-01
2.6344e-01
3.3501e-01
3.1688e-01
3.0541e-01
7.1229e-02
6.1259e-01
3.8157e-01
1.2165e-01
1.9285e-01
9.1869e-03
1.4343e-01
1.9120e-01
5.2493e-01
3.6059e-01
2.3096e-03
6.6493e-02
2.2404e-01
4.1420e-01
1.7576e-01
1.6669e-01
5.0916e-01
2.2352e-01
3.9054e-01
3.9455e-01
6.4627e-01
3.5093e-01
1.2078e-01
6.1916e-01
4.0209e-01
9.0489e-02
7.0811e-01
7.2961e-03
2.2785e-01
9.2322e-02
4.0911e-01
2.6906e-01
2.2416e-02
4.4271e-01
3.5137e-01
3.7948e-01
6.0208e-02
3.6112e-02
9.7509e-01
2.5506e-01
1.1458e+00
1.0578e+00
2.5861e-01
2.2535e-01
1.4216e+00
8.7728e-03
1.3563e-03
3.1390e-01
6.4963e-01
4.3982e-01
1.6805e-01
5.6809e-01
2.2176e-01
2.4469e-01
5.0840e-01
4.4657e-03
2.4578e-01
6.1112e-01
5.3567e-01
3.8226e-01
2.2157e-01
5.1819e-01
5.6275e-02
2.5595e-01
7.8556e-01
2.8737e-01
3.7440e-03
3.3030e-01
2.3840e-01
2.1855e-01
4.6970e-02
2.8357e-01
3.6767e-0

3.5883e-01
3.7116e-01
4.4765e-01
2.8690e-03
6.0685e-01
2.6290e-01
6.5541e-01
1.3092e-01
4.6178e-01
1.1907e-01
2.5217e-01
2.8114e-01
5.2843e-01
1.4115e-01
7.1953e-01
5.1365e-01
1.5959e-01
1.1283e-02
2.0005e-01
9.2553e-02
1.2581e-01
5.1911e-01
2.9903e-01
1.1784e-01
1.7546e-01
5.1401e-01
1.9069e-01
1.7105e-01
3.6422e-01
6.5524e-03
1.4951e-01
4.9359e-01
4.3632e-01
4.3961e-01
4.0389e-01
5.6543e-01
1.0057e-01
4.2823e-01
2.3886e-01
5.8974e-01
1.8184e-03
3.2182e-01
1.4594e-01
1.9633e-01
5.2395e-01
4.3934e-01
3.2082e-01
3.2669e-01
1.0041e-01
1.6447e-02
6.5832e-01
6.9463e-01
3.1680e-01
6.2442e-01
1.4830e-01
5.7386e-01
4.8022e-01
1.2152e-01
2.1856e-01
8.4808e-02
8.3400e-02
1.1286e-01
3.6051e-01
1.9107e-01
1.6356e-01
2.3738e-01
7.3766e-03
6.6847e-01
5.9780e-01
1.4137e-01
2.2623e-01
4.0090e-01
1.3057e-01
1.5054e-01
6.3733e-01
1.4046e-01
4.6476e-02
3.0215e-01
1.1631e-01
3.5196e-01
1.2058e+00
1.3811e-01
1.7856e+00
1.4480e-01
5.3870e-02
4.0121e-01
6.8111e-02
1.2220e-01
1.4109e-01
3.1558e-01
3.2920e-01

2.9286e-01
3.8933e-01
3.7160e-01
1.2199e-01
4.2653e-01
4.8716e-02
9.7434e-02
7.3040e-02
2.9355e-02
1.1950e-01
1.5328e-01
4.0547e-02
1.5081e-01
6.2499e-02
5.6125e-01
2.5281e-01
4.9577e-01
1.5943e-01
4.8255e-01
8.5583e-03
1.1013e-01
3.0500e-01
3.1553e-01
8.4219e-02
1.1423e-01
2.8513e-01
5.3857e-01
2.2442e-01
4.1887e-02
4.2171e-01
5.3979e-01
8.9981e-01
2.8347e-01
2.9620e-01
2.4246e-01
6.5562e-01
1.4256e-01
1.9129e-02
1.5675e+00
8.0166e-02
3.9531e-01
1.7254e-01
2.9438e-01
8.9239e-02
2.1606e-01
4.3350e-01
2.7851e-02
5.0814e-02
2.8370e-01
4.1776e-01
1.2276e-03
5.0396e-02
6.7476e-02
3.5763e-07
3.5083e-01
3.4802e-01
2.7786e-01
2.0399e-01
5.1003e-01
4.8571e-01
5.5140e-02
7.4151e-02
8.4498e-01
1.2365e-04
3.2742e-01
5.2153e-02
3.9825e-01
3.2411e-01
4.4193e-01
5.4288e-01
1.3098e-01
6.2051e-01
3.9785e-01
4.2679e-01
5.1856e-01
3.8344e-01
3.7650e-01
3.3183e-01
3.5607e-01
3.2634e-01
9.0793e-01
3.3893e-01
5.5296e-01
8.7737e-02
5.9556e-01
5.2692e-01
3.7318e-01
6.2420e-01
1.5175e-01
3.4557e-01
1.4959e-02

In [5]:
# This part does the peeler.py operation

mode = "default"
# mode = "no-project" # use it for line-surface

normfpath = os.path.join(path, normfname)
points = fmt.load(normfpath)

m = len(points[0])
print("Peeling data point cloud in {0:s} mode ...".format(mode))
if mode == "default":
    # the defualt mode will project the points 
    # on a simplex on the first quadrant
    ppoints = plr.project(points)
    cpoints = plr.collapse(ppoints, dim = m - 1)
    boundaries = plr.peel(cpoints)
elif mode == "no-project":
    boundaries = plr.peel(points)
fmt.cat(boundaries, dtype = 'int')

layerfpath = os.path.join(path, normfname.split('.')[0] + "-layers.out")
print("Saving {0:d} layers into {1:s} ...".format(len(boundaries), layerfpath))
fmt.save(boundaries, layerfpath, dtype = 'int')

Peeling data point cloud in default mode ...
Total points: 10679
Computing alpha-shape ...
alpha-shape done.
Layer 0: points added 6876, points left 3803.
Computing alpha-shape ...
alpha-shape done.
Layer 1: points added 2721, points left 1082.
Computing alpha-shape ...
alpha-shape done.
Layer 2: points added 936, points left 146.
Computing alpha-shape ...
alpha-shape done.
Layer 3: points added 141, points left 5.
Layer 4: points added 5. 

0	3	5	6	11	12	14	15	17	19	21	22	23	24	25	26	27	29	31	32	33	34	36	37	39	40	41	45	46	48	49	50	51	52	54	55	59	60	62	63	64	66	67	68	69	70	72	75	76	78	79	80	81	82	83	85	87	88	89	90	93	97	100	102	105	108	110	111	112	113	114	118	120	121	122	123	124	127	128	129	130	132	137	139	140	141	142	143	144	145	146	148	152	153	155	156	157	158	160	161	162	164	165	166	167	168	172	173	174	175	176	177	178	181	182	183	184	185	186	187	189	190	191	198	199	201	202	203	204	206	207	208	209	211	213	214	215	216	217	218	220	223	224	225	226	227	228	232	233	234	235	236	242	243	244	

In [7]:
# This part does the palettizer.py operation

n_layers = 4
mode = "invsc"
# mode = "invrv"
# mode = "rv"
# mode = "sc"

normfpath = os.path.join(path, normfname)
points = fmt.load(normfpath)

layerfile = normfname.split('.')[0] + "-layers.out"
layers = fmt.load(os.path.join(path, layerfile), dtype = 'int')

print("Computing palette coordinates with {0:d} layers in {1:s} mode ...".format(n_layers, mode))
if mode == "invsc":
    palette_coords = pltz.palettize_sc(points, layers, n_layers = n_layers, inverted = True)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-invsc.out")
elif mode == "invrv":
    palette_coords = pltz.palettize_rv(points, layers, n_layers = n_layers, inverted = True)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-invrv.out")
elif mode == "rv":
    palette_coords = pltz.palettize_rv(points, layers, n_layers = n_layers)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-rv.out")
elif mode == "sc":
    palette_coords = pltz.palettize_sc(points, layers, n_layers = n_layers)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-sc.out")
else:
    print("Error: unknown mode \'{0:s}\'\n".format(mode))
    sys.exit(1)

print("Saving palette coordinates into {0:s} ...".format(palettefpath))
pltz.save_palette(palette_coords, palettefpath)

Computing palette coordinates with 4 layers in invsc mode ...
Saving palette coordinates into ../data/isolated/isolated-8d-norm-palette-invsc.out ...
